In [1]:
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects import numpy2ri
from rpy2.robjects.packages import importr

# Activate automatic conversion between R and NumPy
numpy2ri.activate()

# Import necessary R packages
base = importr('base')
psych = importr('psych')

def pareto_scale(X):
    """Pareto scaling: Center and scale by sqrt of standard deviation."""
    mean_X = np.mean(X, axis=0, keepdims=True)
    std_X = np.std(X, axis=0, keepdims=True)
    return (X - mean_X) / np.sqrt(std_X)

def pca_varimax(X, n_components):
    """Perform PCA with Varimax rotation using R's psych package."""
    # Scale and center the data
    X_scaled = pareto_scale(X)
    
    # Convert to R matrix
    X_r = ro.r.matrix(X_scaled, nrow=X_scaled.shape[0], ncol=X_scaled.shape[1])

    # Perform PCA with Varimax rotation
    pca_res = psych.principal(X_r, nfactors=n_components, rotate="varimax")

    # Extract rotated loadings and scores
    loadings = np.array(pca_res.rx2('loadings'))  # Factor loadings
    scores = np.array(pca_res.rx2('scores'))      # Rotated scores

    return loadings, scores


In [6]:
import numpy as np
from sklearn.decomposition import FactorAnalysis

def fa_varimax(X, n_components):
    """Perform Factor Analysis with built-in Varimax rotation using sklearn."""
    X_scaled = pareto_scale(X) # Transpose to match scikit-learn’s input shape

    # Perform Factor Analysis with Varimax rotation
    fa = FactorAnalysis(n_components=n_components, rotation="varimax")
    scores = fa.fit_transform(X_scaled)  # Factor scores
    loadings = fa.components_.T # Factor loadings

    return loadings, scores


In [4]:
def _ortho_rotation(components, method="varimax", tol=1e-6, max_iter=100):
  """Return rotated components."""
  nrow, ncol = components.shape
  rotation_matrix = np.eye(ncol)
  var = 0

  for _ in range(max_iter):
    comp_rot = np.dot(components, rotation_matrix)
    if method == "varimax":
      tmp = comp_rot * np.transpose((comp_rot**2).sum(axis=0) / nrow)
    elif method == "quartimax":
      tmp = 0
    u, s, v = np.linalg.svd(np.dot(components.T, comp_rot**3 - tmp))
    rotation_matrix = np.dot(u, v)
    var_new = np.sum(s)
    if var != 0 and var_new < var * (1 + tol):
      break
    var = var_new

  return np.dot(components, rotation_matrix).T

In [20]:

# Example usage:
np.random.seed(42)
X = np.random.rand(5,100)
X_scaled = pareto_scale(X) # Transpose to match scikit-learn’s input shape
loadings_r, scores_r = pca_varimax(X, n_components=2)
loadings, scores = fa_varimax(X, n_components=2)
loadings2 = _ortho_rotation(X_scaled, method="varimax", tol=1e-6, max_iter=100)


U, s, _ = np.linalg.svd(X)
U.shape

# print("Loadings:\n", loadings, loadings_r, loadings2[:,:2])
# print("Scores:\n", scores, scores_r)

(5, 5)

In [15]:
loadings_r @ loadings2[:,:2].T

array([[-0.15992177,  0.17066168,  0.0413745 , -0.42969206,  0.62351175],
       [ 0.13882294,  0.17501659, -0.299314  ,  0.25363725, -0.51716958],
       [ 0.16958953,  0.07533691, -0.25278936,  0.36099451, -0.64210526],
       [-0.11400892, -0.66993577,  0.67470173, -0.01394033,  0.38551698],
       [-0.07230984, -0.79768581,  0.731768  ,  0.12885024,  0.21673479]])

In [ ]:
def _rotate(components, n_components=None, tol=1e-6):
  "Rotate the factor analysis solution."
  # note that tol is not exposed
  return _ortho_rotation(components.T, method="varimax", tol=tol)
  ]

In [17]:
import numpy as np
from sklearn.decomposition import FactorAnalysis, PCA
from scipy.linalg import svd

def pareto_scale(X):
    """Pareto scaling: Center and scale by sqrt of standard deviation."""
    mean_X = np.mean(X, axis=1, keepdims=True)
    std_X = np.std(X, axis=1, keepdims=True)
    return (X - mean_X) / np.sqrt(std_X)

def varimax(Phi, gamma=1.0, q=20, tol=1e-6):
    """Manual Varimax Rotation."""
    p, k = Phi.shape
    R = np.eye(k)
    d = 0

    for _ in range(q):
        Lambda = np.dot(Phi, R)
        u, s, vh = svd(np.dot(Phi.T, Lambda**3 - (gamma / p) * np.dot(Lambda, np.diag(np.sum(Lambda**2, axis=0)))))
        R = np.dot(u, vh)
        d_new = np.sum(s)
        if d != 0 and d_new / d < 1 + tol:
            break
        d = d_new

    return np.dot(Phi, R)

def pca_varimax(X, n_components):
    """PCA followed by manual Varimax rotation."""
    X_scaled = pareto_scale(X).T

    # PCA
    pca = PCA(n_components=n_components)
    scores = pca.fit_transform(X_scaled)
    loadings = pca.components_.T

    # Apply manual Varimax rotation
    rotated_loadings = varimax(loadings)

    return rotated_loadings, scores

def fa_varimax(X, n_components):
    """Factor Analysis with built-in Varimax."""
    X_scaled = pareto_scale(X).T

    # Factor Analysis with Varimax
    fa = FactorAnalysis(n_components=n_components, rotation="varimax")
    scores = fa.fit_transform(X_scaled)
    loadings = fa.components_.T

    return loadings, scores

# Generate Example Data
np.random.seed(42)
X = np.random.rand(5, 100)  # (5 variables, 100 measurements)

# Compare PCA + Varimax vs. FA + Varimax
pca_loadings, pca_scores = pca_varimax(X, n_components=2)
fa_loadings, fa_scores = fa_varimax(X, n_components=2)

# Print Results
print("\nPCA + Manual Varimax Loadings:\n", pca_loadings)
print("\nFactorAnalysis + Built-in Varimax Loadings:\n", fa_loadings)

print("\nAbsolute Difference in Loadings:\n", np.abs(pca_loadings - fa_loadings))





PCA + Manual Varimax Loadings:
 [[-0.43046584  0.39807482]
 [ 0.27746008 -0.38647872]
 [-0.52390574 -0.04157466]
 [ 0.6669407   0.22207964]
 [ 0.13574957  0.80070146]]

FactorAnalysis + Built-in Varimax Loadings:
 [[-0.15469816  0.16656794]
 [ 0.01012642 -0.07625491]
 [-0.13394149 -0.0136996 ]
 [ 0.42249017  0.04166929]
 [ 0.10152476  0.3123162 ]]

Absolute Difference in Loadings:
 [[0.27576768 0.23150688]
 [0.26733365 0.3102238 ]
 [0.38996426 0.02787506]
 [0.24445053 0.18041034]
 [0.03422481 0.48838525]]
